In [1]:
# IMPORTS
import tensorflow as tf
import numpy as np
import random
from PIL import Image
from tensorflow.python.keras.models import Input, Model
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, concatenate, Dense, Conv3D, MaxPool3D, Flatten
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 

In [2]:
# CONSTANTS
BSE_SHAPE = (500, 500, 1)
EBSD_SHAPE = (200, 200, 3)
OM_SHAPE = (1000, 1000, 1)


In [3]:
# VARIABLES
CONV_NODES = [64, 128, 256, 512]
DENSE_NODES = [32, 64, 128]

NUM_CONV = [0, 1]
NUM_DENSE = [0, 1, 2]


In [4]:
# TRAIN DATA IMPORT
BSE = 'path'
EBSD = 'path'
OM = 'path'


In [5]:
# # #### testing network ####
# test_path = 'test_data/'
# 
# # load test img
# i = load_img('test_data/0/bse.png')
# x = img_to_array(i)


In [6]:
# #### Build Model ####
# INPUTS
input_1 = Input(shape=(100,100,3))
input_2 = Input(shape=(100,100,3))
input_3 = Input(shape=(100,100,3))

# MODEL

# FIRST BRANCH
# BSE
# needs to accept a black and white image
one = Conv2D(64, (3, 3), activation="relu")(input_1)
one = MaxPool2D(pool_size=(2, 2))(one)

one = Conv2D(64, 3, activation='relu')(one)
one = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(one)

one = Flatten()(one)

one = Model(inputs=input_1, outputs=one)

# SECOND BRANCH
# EBSD
# needs to accept a RGB image
two = Conv2D(64, (3, 3), activation='relu')(input_2)
two = MaxPool2D(pool_size=(2, 2))(two)

two = Conv2D(64, 3, activation='relu')(two)
two = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(two)

two = Flatten()(two)

two = Model(inputs=input_2, outputs=two)

# THIRD BRANCH
# OM
# needs to accept a black and white image
three = Conv2D(64, (3, 3), activation='relu')(input_3)
three = MaxPool2D(pool_size=(2, 2))(three)

three = Conv2D(64, 3, activation='relu')(three)
three = MaxPool2D(pool_size=(2, 2), strides=2, padding='valid')(three)

three = Flatten()(three)

three = Model(inputs=input_3, outputs=three)

# COMBINE THE BRANCHES
# concatenates the three branches all together into one
combined = concatenate([one.output, two.output])
combined = concatenate([combined, three.output])

# END BRANCH
end = Dense(64, activation='relu')(combined)
end = Dense(1, activation='sigmoid')(end)

# MODEL
model = Model(inputs=[one.input, two.input, three.input], outputs=end)

# COMPILE MODEL
model.compile(loss='poisson', optimizer='adam', metrics=['cosine_proximity'])



In [7]:
# IMPORT DATA

BATCH_SIZE = 200

# load in data as an array of arrays which contain [bse, ebsd, om, stress]
train_data = []
for i in range(int(BATCH_SIZE / 2)):
    path = 'test_data/SR/train/' + str(i) + '/'
    bse = load_img(path + 'bse.png')
    bse = np.array(bse.resize((100, 100))).ravel()
    # bse = tf.keras.utils.normalize(bse, axis=1)
    
    ebsd = load_img(path + 'ebsd.png')
    ebsd = np.array(ebsd.resize((100, 100))).ravel()
    # ebsd = tf.keras.utils.normalize(ebsd, axis=1)
    
    om = load_img(path + 'om.png')
    om = np.array(om.resize((100, 100))).ravel()
    # om = tf.keras.utils.normalize(om, axis=1)
    
    train_data.append([bse, ebsd, om, 1060])
    
for i in range(int(BATCH_SIZE / 2)):
    path = 'test_data/HT/train/' + str(i) + '/'
    bse = load_img(path + 'bse.png')
    bse = np.array(bse.resize((100, 100))).ravel()
    # bse = tf.keras.utils.normalize(bse, axis=1)
    
    ebsd = load_img(path + 'ebsd.png')
    ebsd = np.array(ebsd.resize((100, 100))).ravel()
    # ebsd = tf.keras.utils.normalize(ebsd, axis=1)
    
    om = load_img(path + 'om.png')
    om = np.array(om.resize((100, 100))).ravel()
    # om = tf.keras.utils.normalize(om, axis=1)
    
    train_data.append([bse, ebsd, om, 918])

# shuffle the data array
train_data = np.array(train_data)
random.shuffle(train_data)

# load data into input data
bse_train = []
ebsd_train = []
om_train = []
stress_train = []

for i in range(len(train_data)):
    bse_train.append(train_data[i][0])
    ebsd_train.append(train_data[i][1])
    om_train.append(train_data[i][2])
    stress_train.append(train_data[i][3])


# reshape the arrays of each thing
bse_train = np.array(bse_train)
bse_train = bse_train.reshape(BATCH_SIZE, 100, 100, 3)

ebsd_train = np.array(ebsd_train)
ebsd_train = ebsd_train.reshape(BATCH_SIZE, 100, 100, 3)

om_train = np.array(om_train)
om_train = om_train.reshape(BATCH_SIZE, 100, 100, 3)

stress_train = np.array(stress_train)
stress_train = stress_train.reshape(BATCH_SIZE, 1)

In [9]:
# FIT MODEL
# callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='.\Graph', histogram_freq=0,
                                                      write_graph=True, write_images=True)

model.fit([bse_train, ebsd_train, om_train],
          [stress_train], 
          epochs = 10,
          validation_split=0.2, 
          callbacks=[tensorboard_callback])


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 0s 3ms/sample - loss: 0.9999 - cosine_proximity: 1.0000 - val_loss: 0.9999 - val_cosine_proximity: 1.0000
Epoch 2/10
160/160 [==============================] - 0s 1ms/sample - loss: 0.9999 - cosine_proximity: 1.0000 - val_loss: 0.9999 - val_cosine_proximity: 1.0000
Epoch 3/10
160/160 [==============================] - 0s 1ms/sample - loss: 0.9999 - cosine_proximity: 1.0000 - val_loss: 0.9999 - val_cosine_proximity: 1.0000
Epoch 4/10
160/160 [==============================] - 0s 1ms/sample - loss: 0.9999 - cosine_proximity: 1.0000 - val_loss: 0.9999 - val_cosine_proximity: 1.0000
Epoch 5/10
160/160 [==============================] - 0s 1ms/sample - loss: 0.9999 - cosine_proximity: 1.0000 - val_loss: 0.9999 - val_cosine_proximity: 1.0000
Epoch 6/10
160/160 [==============================] - 0s 1ms/sample - loss: 0.9999 - cosine_proximity: 1.0000 - val_loss: 0.9999 - val_cosine_proximity: 1.